# Homework 1 - Deep Learning Winter 2024
TA: Jerry Abu Ayoub

<font color='aqua'>
Student 1: Or Dinar 207035809
</font>
<br>
<font color='aqua'>
Student 2: Ramiz Srour 314855446
</font>

## Libraries and Drive

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset, random_split
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),                  # convert transforms to tensors
    transforms.Normalize((0.5,), (0.5,))    # normalization
])

# training set
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True
)

# test set
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    transform=transform,
    download=True
)

# Define the labels to keep
labels_to_keep = [0, 1, 2, 3]

# Filter the training set
train_indices = torch.isin(train_dataset.targets, torch.tensor(labels_to_keep))
X_train, y_train = train_dataset.data[train_indices], train_dataset.targets[train_indices]

# Filter the test set
test_indices = torch.isin(test_dataset.targets, torch.tensor(labels_to_keep))
X_test, y_test = test_dataset.data[test_indices], test_dataset.targets[test_indices]

In [3]:
class Network1(nn.Module):
    def __init__(self, input_size=784, output_size=10):
        super(Network1, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)  # First layer

    def forward(self, x):
        x = self.fc1(x)
        return x

In [4]:
model1 = Network1()
param_count1 = sum(p.numel() for p in model1.parameters() if p.requires_grad)
print(f'Number of parameters in model 1: {param_count1}')

Number of parameters in model 1: 7850


In [5]:
X_train = X_train.float()
X_train = X_train.view(X_train.size(0), -1)

X_test = X_test.float()
X_test = X_test.view(X_test.size(0), -1)

y_train = y_train.long()
y_test = y_test.long()

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

In [7]:
from torch.utils.data import DataLoader, TensorDataset

# Create a DataLoader for the training set
batch_size = 1024  # Define your batch size
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
num_epochs = 150

for epoch in range(num_epochs):
    model1.train()
    epoch_loss = 0.0

    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model1(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss for the epoch
        epoch_loss += loss.item()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}')

print("Training complete")


Epoch [1/150], Loss: 15.6429
Epoch [11/150], Loss: 1.4968
Epoch [21/150], Loss: 0.9672
Epoch [31/150], Loss: 0.6532
Epoch [41/150], Loss: 0.5705
Epoch [51/150], Loss: 0.5950
Epoch [61/150], Loss: 0.9023
Epoch [71/150], Loss: 0.5457
Epoch [81/150], Loss: 0.4569
Epoch [91/150], Loss: 0.6009
Epoch [101/150], Loss: 0.4058
Epoch [111/150], Loss: 0.4268
Epoch [121/150], Loss: 0.4885
Epoch [131/150], Loss: 0.3463
Epoch [141/150], Loss: 0.7104
Training complete


In [8]:
model1.eval()

# disable gradient calculation
with torch.no_grad():
    outputs = model1(X_test)

    _, predictions = torch.max(outputs, 1)

    correct = (predictions == y_test).sum().item()
    total = y_test.size(0)
    accuracy = correct / total

print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 90.25%
